In [1]:
_HIGH_ = 'High'
dty = {   
        'Close': 'float64',
        _HIGH_: 'float64',
        'Low': 'float64',
        'Open': 'float64',
        'Volume': 'int64'
      }

def foo (subset):
    # return a dict
    return {'foo' : subset.size}

ai = {'foo_call': {
                    'name' : 'foo_test',
                    'frequency' : 2,
                    'callback': foo,
                    'params' : {_HIGH_:dty[_HIGH_], 'Low':dty['Low']},   # axis(ohlcv) 
                    'ext_data' : {} # globals (min,max, histvol, TBD)
                  }
      }
ai

{'foo_call': {'callback': <function __main__.foo>,
  'ext_data': {},
  'frequency': 2,
  'name': 'foo_test',
  'params': {'High': 'float64', 'Low': 'float64'}}}

In [2]:
max_duration = max({_:value['frequency'] for (_,value) in ai.items()}.values())
max_duration

2

In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
pdf = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/test-ai.csv' \
#pdf = pd.read_csv('/home/starq/REP/DATA/FINANCE/Quotazioni/NVDA.csv' \
                                 ,usecols=['Date','Open', 'High', 'Low', 'Close', 'Volume'] \
                                 ,dtype=dty,parse_dates=['Date'],infer_datetime_format=True \
                                 ,index_col='Date')
pdf.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12 entries, 1999-01-22 to 1999-02-08
Data columns (total 5 columns):
Open      12 non-null float64
High      12 non-null float64
Low       12 non-null float64
Close     12 non-null float64
Volume    12 non-null int64
dtypes: float64(4), int64(1)
memory usage: 576.0 bytes


In [5]:
def stack_execution (active_indicators, datapoint, dtype=dty, ext_data={}):
    
    ai           = active_indicators 
    max_duration = max({_:value['frequency'] for (_,value) in ai.items()}.values())
    i_stack      = pd.DataFrame()
    i_stack      = i_stack.append([datapoint], ignore_index=True)
    ext_data     = ext_data

    period = max_duration
    count  = 1
    print(i_stack)
    
    def _run(pdf): ### può essere un rif a pdf

        nonlocal ai, i_stack, period, count, ext_data

        #i_stack  = i_stack.append([datapoint], ignore_index=True) ### nn serve più
        count += 1
        #print(i_stack)
        
        v = None
        events = list()  ### dovrebbe essere un dict ??
        
        for fx in ai:
            
            if (count >= ai[fx]['frequency']):
                #
                pass
                #row_list    = -fx['frequency':]  ### si usa sotto

                v = ai[fx]['callback'] (pdf.iloc[-ai[fx]['frequency']:]) ### ultimi fx['frequency'] elementi
                #
                # v potrebbe essere :
                # {'LW-min' : True }

                #v = ai[fx]['callback'] ()
                # i_stack[row_list, :][:, columns_list]
                #v = fx['callback'] (i_stack[np.ix_(row_list, columns_list)]) ### best solution x nparray (fare un performance test)
                              
                #events.append[{str(ai[fx]['name']):v}]
                # LIST
                events.append(v) ### events.append(v, timestamp) ?
                # DICT
                #events[v.key] = v.value
                
        if count >= period: 
            pass
            #i_stack = i_stack.drop(i_stack.index[0])   ### non serve più
        return events
        #
        # events (LIST) potrebbe essere:
        # [{'LW-min' : True }, {'GAP' : '1.45' }]
        # oppure (DICT)
        # {'LW-min' : True, 'GAP' : '1.45' }
        #
        #return v
        
    return _run

In [6]:
pdf_ext_data_columns = {
                        '0' : 'LW-min',
                        '1' : 'LW-max',
                        '2' : 'GAP',
                        '3' : 'foo'
                        }

# events normalization:
def yeld_event_row (events):

    pdf_ext_data = {}
    if not events:
        for i in range (len(pdf_ext_data_columns)):
            pdf_ext_data [pdf_ext_data_columns[str(i)]] = None
        return pdf_ext_data
        
    for i in range (len(pdf_ext_data_columns)): ### pdf_mapped_events

        #
        # crea l'event_dict (pdf_ext_data) x il datapoint corrente
        #
        try:
            if events[pdf_ext_data_columns[str(i)]]:
                pdf_ext_data [pdf_ext_data_columns[str(i)]] = events[pdf_ext_data_columns[str(i)]]
            else:
                pdf_ext_data [pdf_ext_data_columns[str(i)]] = None
        except KeyError as e:
            pdf_ext_data [pdf_ext_data_columns[str(i)]] = None

    return pdf_ext_data
    #
    # pdf_ext_data potrebbe essere :
    # {'LW-min': None, 'LW-max': None, 'GAP': '1.45'}
        
#
#
#pdf.insert(loc=0, column=pdf_ext_data_columns.values, value=pd.Series(pdf_ext_data))

In [7]:
#
#  MAIN
#

rows = 0
events_rows = list()
start = time.clock()
for datapoint in pdf.itertuples():
    if rows :
        #break
        events = run_indicators(pdf) ### aggiungere datapoint.timestamp come idx ?
        if events:
            print ('events :')
            print(events)
            pass ##########################################################################
            #
            # events normalization:
            #
            # ...qui bisogna - con la lista di tutte colonne utilizzate in ai (pdf_ext_data_columns) -
            # verificare se ogni colonna esiste in events, se non esiste la crea vuota in pdf_ext_data
            # se esiste la copia da events            
            events_dict = yeld_event_row(events)
            #
            # quindi :
            events_rows.append(events_dict)
            #
            #################################################################################           
            
        else:
            events_rows.append(yeld_event_row({}))
    else:
        run_indicators = stack_execution(ai, datapoint)  
        ### events_row.append(_null_event_row_)
        events_rows.append(yeld_event_row({}))

    rows += 1

columns =  list(pdf_ext_data_columns.values())
pdf.insert(loc=0, column=columns, value=event_stack)
#pdf.insert(loc=0, column=pdf_ext_data_columns.<values>, value=pd.Series(pdf_ext_data))

# provare :
# https://stackoverflow.com/questions/10715965/add-one-row-in-a-pandas-dataframe
#
'''
numOfRows = 10000
row_list = []

for i in range( 1,numOfRows):
    dict1 = dict( (a,numpy.random.randint(100)) for a in ['A','B','C','D','E'])
    event_stack.append(dict1)

df3 = pandas.DataFrame(event_stack, columns=['A','B','C','D','E']) ### qui devo aggiungere a pdf!!!

# quindi :
https://pandas.pydata.org/pandas-docs/stable/merging.html !!!!
'''    

elapsed = str(time.clock() - start)
print('elapsed : ' + elapsed)


       Index  Open      High       Low     Close    Volume
0 1999-01-22  21.0  23.43744  18.62496  19.68744  67867200
events :
[{'foo': 10}]


TypeError: list indices must be integers or slices, not str

In [37]:
np.__version__

'1.11.1'

In [109]:
xlist = [(1, 1.1), (2, 1.2), (3, 1.3)]
#xlist = list(datapoint)
#dt=np.dtype('Timestamp,float,float,float,float,int')
xarr = np.array(xlist)#,dtype=dt)
xlist
#xarr
#xarr.dtype
list(datapoint)
xarr = np.array(list(datapoint))
print(xarr)

[Timestamp('1999-01-25 00:00:00') 21.249960000000002 21.999960000000002
 19.687439999999999 21.75 12762000]


In [305]:
dp = [1.99,2,3,4]
print(type(dp))
print(dp)
arr = np.array(dp)#[[1,2,3,4], [5,6,7,8], [9,10,11,12]])
arr

<class 'list'>
[1.99, 2, 3, 4]


array([ 1.99,  2.  ,  3.  ,  4.  ])

In [250]:
arr[1]
#arr = np.delete(arr, 0,0)

array([5, 6, 7, 8])

In [176]:
arr

array([[ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [8]:
list(pdf_ext_data_columns.values())

['foo', 'LW-max', 'GAP', 'LW-min']